In [1]:
from tokenizer import CharTokenizer
from model import GPT
from model.config import GPTConfig, config_for

In [2]:
with open("resources/pushkin.txt", "r") as f:
    lines = f.readlines()

text = " ".join(lines)
print(text)

tokenizer = CharTokenizer.train([text])

Любви, надежды, тихой славы
 Недолго нежил нас обман,
 Исчезли юные забавы,
 Как сон, как утренний туман;
 Но в нас горит еще желанье,
 Под гнетом власти роковой
 Нетерпеливою душой
 Отчизны внемлем призыванье.
 Мы ждем с томленьем упованья
 Минуты вольности святой,
 Как ждет любовник молодой
 Минуты верного свиданья.
 Пока свободою горим,
 Пока сердца для чести живы,
 Мой друг, отчизне посвятим
 Души прекрасные порывы!
 Товарищ, верь: взойдет она,
 Звезда пленительного счастья,
 Россия вспрянет ото сна,
 И на обломках самовластья
 Напишут наши имена!


In [3]:
# config = config_for("tiny", vocab_size=len(tokenizer.vocab), dropout=0.2)
config = GPTConfig(
    vocab_size=tokenizer.vocab_length(),
    num_decoder_layers=4,
    dim_feedforward=768,
    embedding_dim=384,
    num_decoder_heads=6,
    decoder_head_dim=64,
    dropout=0.5,
    max_seq_len=256
)

config

GPTConfig(vocab_size=36, num_decoder_layers=4, embedding_dim=384, dim_feedforward=768, num_decoder_heads=6, decoder_head_dim=64, max_seq_len=256, dropout=0.5)

In [4]:
tokenizer.save("fuck.tokenizer.json")

In [5]:
gpt = GPT(config)
gpt

GPT(
  (tok_embeds): Embedding(36, 384)
  (decoder): GPTDecoder(
    (decoders): ModuleList(
      (0): GPTDecoderBlock(
        (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (ln2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (mha): MultiHeadAttention(
          (heads): ModuleList(
            (0): AttentionHead(head_dim=64)
            (1): AttentionHead(head_dim=64)
            (2): AttentionHead(head_dim=64)
            (3): AttentionHead(head_dim=64)
            (4): AttentionHead(head_dim=64)
            (5): AttentionHead(head_dim=64)
          )
          (projection): Sequential(
            (0): Linear(in_features=384, out_features=384, bias=True)
            (1): Dropout(p=0.5, inplace=False)
          )
        )
        (mlp): FeedForward(
          (ff): Sequential(
            (0): Linear(in_features=384, out_features=768, bias=True)
            (1): GoogleGELU()
            (2): Linear(in_features=768, out_features=384, bias=True

In [6]:
encoded = tokenizer.encode(text)
len(encoded)

557

In [7]:
import torch

BATCH_SIZE = 256
BATCHES = len(encoded) // BATCH_SIZE
EPOCHS = 100

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(gpt.parameters(), lr=1e-3)

In [8]:
for e in range(EPOCHS):
    epoch_loss = 0
    for i in range(BATCHES):
        optimizer.zero_grad()
        x = encoded[i * BATCH_SIZE: (i + 1) * BATCH_SIZE]
        y = encoded[i * BATCH_SIZE + 1: (i + 1) * BATCH_SIZE + 1]

        x = torch.LongTensor(x).unsqueeze(0)
        y = torch.LongTensor(y).unsqueeze(0)
        m = torch.tril(torch.ones(BATCH_SIZE, BATCH_SIZE))

        output = gpt(x, attention_mask=m, output_attentions=True)

        loss = criterion(output.logits.transpose(1, 2), y)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"epoch {e + 1} loss: {epoch_loss / BATCHES}")

epoch 1 loss: 3.8048110008239746
epoch 2 loss: 2.9936805963516235
epoch 3 loss: 2.6144509315490723
epoch 4 loss: 2.3087966442108154
epoch 5 loss: 2.018452823162079
epoch 6 loss: 1.7976194620132446
epoch 7 loss: 1.6444581747055054
epoch 8 loss: 1.4550820589065552
epoch 9 loss: 1.2621997594833374
epoch 10 loss: 1.0676565170288086
epoch 11 loss: 0.8886569738388062
epoch 12 loss: 0.7775632441043854
epoch 13 loss: 0.5741545557975769
epoch 14 loss: 0.47391851246356964
epoch 15 loss: 0.38171617686748505
epoch 16 loss: 0.3044995069503784
epoch 17 loss: 0.2503356635570526
epoch 18 loss: 0.19674308598041534
epoch 19 loss: 0.16712970286607742
epoch 20 loss: 0.14521539211273193
epoch 21 loss: 0.1163438968360424
epoch 22 loss: 0.1030660904943943
epoch 23 loss: 0.08055250719189644
epoch 24 loss: 0.07905955240130424
epoch 25 loss: 0.06037338450551033
epoch 26 loss: 0.06072022579610348
epoch 27 loss: 0.045732857659459114
epoch 28 loss: 0.05219606123864651
epoch 29 loss: 0.04405752569437027
epoch 30 lo

In [13]:
text = "исчезли "

iters = 100

for _ in range(iters):
    input_ids = torch.tensor(tokenizer.encode([text[-BATCH_SIZE:]]))
    with torch.no_grad():
        gpt.eval()
        logits = gpt(input_ids).logits
        logits = logits[0, -1]

        token_id = logits.argmax().item()

        text += tokenizer.id_to_token(token_id)

print(text)

исчезли сдежды, тихой славы
 недолго нежил нас обман,
 исчезли юные забавы,
 как сон, как утренний туман;
 н
